In [ ]:
# from operator import index
import pandas as pd
import numpy as np
import os
import splitfolders
import torch
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def splitting(path):
    if os.path.exists("data/dataset_splits"):
        return
    else:
        splitfolders.ratio(path, output="data/dataset_splits", seed=1337, ratio=(.8, 0.1,0.1))



def dataset_loader(train_path, val_path, test_path):
    train_transform = transforms.Compose([transforms.Resize(224),
                                        transforms.RandomRotation(30),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

    val_transform = transforms.Compose([transforms.Resize(224, 224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

    test_transform = transforms.Compose([transforms.Resize(224, 224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])])

    train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
    val_dataset = datasets.ImageFolder(val_path, transform=val_transform)
    test_dataset = datasets.ImageFolder(test_path, transform=test_transform)
    print(f"Training data samples:{len(train_dataset)}\nValidation data samples:\
         {len(val_dataset)}\nTesting data samples: {len(test_dataset)}")

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

    print('Data DUELY HANDLED')

    return train_loader, val_loader, test_loader

In [ ]:
train_loader, val_loader, test_loader = dataset_loader("data/dataset_splits/train", "data/dataset_splits/val", "data/dataset_splits/test")

In [ ]:
images, labels = iter(train_loader).next()

In [ ]:
import matplotlib.pyplot as plt
import torchvision

In [ ]:
class_names = [cat for cat in os.listdir("data/2750")]
class_names

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

plt.figure(figsize=(20,10))
imshow(out, title=[class_names[x] for x in classes])

In [ ]:
from torch import nn
import torch.nn.functional as F
from torchvision import models

In [ ]:
class SatImgNet(nn.Module):
    def __init__(self):
        super(SatImgNet, self).__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
model.fc = SatImgNet()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')